<a href="https://colab.research.google.com/github/BRA7534/DeepSeek-Coder/blob/main/DeepSeek_Coder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Charger le modèle directement depuis Hugging Face
model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

def generate_code(prompt):
    # Encoder l'entrée
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Générer une réponse
    outputs = model.generate(**inputs, max_length=512)

    # Décoder et retourner la réponse
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Exemple d'utilisation
if __name__ == "__main__":
    while True:
        prompt = input("Vous: ")
        if prompt.lower() in ["exit", "quit"]:
            break
        response = generate_code(prompt)
        print("DeepSeek Coder:", response)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

...

Vous: Write a Python function to calculate the factorial of a number.


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


DeepSeek Coder: Write a Python function to calculate the factorial of a number.


def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))

# Output: 120

# This function works by recursively calling itself with the argument decreased by 1, until it reaches 0. At each step, it multiplies the current value of n with the result of the recursive call. The base case for the recursion is when n is 0, in which case the function returns 1.
<jupyter_output>
120
<jupyter_text>
Question 2:Write a Python function to calculate the Fibonacci series up to n.
<jupyter_code>
def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

print(fibonacci(10))

# Output: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

# This function starts with a list containing the first two numbers in the Fibonacci sequence (0 and 1). It then enters a loop that contin

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


DeepSeek Coder: parles tu en francais ?


## intent:greet
- Bonjour
- Bonsoir
- Hello
- Hola
- Hey
- Salut
- Bounjour
- Guten Tag
- Dia dhuit
- Zdravstvuyte
- Dobr� den
- Hej
- Aloha
- Ciao
- Hola
- Hei
- Hallo
- Hey
- Hei
- Hej
- Hallå
- Hei
- Hej
- Hei
- Hey
- Hola
- Hello
- Bonjour
- Guten Tag
- Guten Morgen
- Bounjour
- Dobr� den
- Zdravstvuyte
- Dia dhuit
- Hola
- Hej
- Hei
- Hallo
- Hey
- Hei
- Hej
- Hallå
- Hei
- Hej
- Hei
- Hey
- Hola
- Hello
- Bonjour
- Guten Tag
- Guten Morgen
- Bounjour
- Dobr� den
- Zdravstvuyte
- Dia dhuit
- Hola
- Hej
- Hei
- Hallo
- Hey
- Hei
- Hej
- Hallå
- Hei
- Hej
- Hei
- Hey
- Hola
- Hello
- Bonjour
- Guten Tag
- Guten Morgen
- Bounjour
- Dobr� den
- Zdravstvuyte
- Dia dhuit
- Hola
- Hej
- Hei
- Hallo
- Hey
- Hei
- Hej
- Hallå
- Hei
- Hej
- Hei
- Hey
- Hola
- Hello
- Bonjour
- Guten Tag
- Guten Morgen
- Bounjour
- Dobr� den
- Zdravstvuyte
- Dia dhuit
- Hola
- Hej
- Hei
- Hallo
- Hey
- Hei
- Hej
